In [1]:
import numpy as np
import pandas as pd
import nltk
import re
import matplotlib.pyplot as plt
from json import JSONDecoder
from functools import partial
import json
from pprint import pprint

In [ ]:
# def json_parse(fileobj, decoder=JSONDecoder(), buffersize=2048):
#     buffer = ''
#     for chunk in iter(partial(fileobj.read, buffersize), ''):
#         buffer += chunk
#         while buffer:
#             try:
#                 result, index = decoder.raw_decode(buffer)
#                 yield result
#                 buffer = buffer[index:]
#             except ValueError:
#                 # Not enough data to decode, read more
#                 break
                
# filename = '../datasets/av-news/first_av_crash.json'

# # for loading the data if saved in a list, not memory efficient
# # with open(filename, 'r') as content:
# #     data = json.load(content)
# # data = data[0]

# # more memory efficient if not saved in a list
# with open(filename, 'r') as content:
#     for data in json_parse(content):
#         break

# Save the relevant attributes in a table format
# def saveAsTable(data):
#     results = []
#     for key, value in data.items():
#         tweetid = key
#         tweet = value[0]
#         author = value[1]
#         user_dest = value[2]
#         date = value[3]
#         depth = value[4].split(' = ')[1]
#         parent_tweetid = value[5]
        
#         result = [tweet, tweetid, date, depth, author, user_dest, parent_tweetid]
#         results.append(result)
        
#     results = pd.DataFrame(results, columns=['Tweet', 'TweetID', 'Date', 'Depth', 'Author', 'UserDest', 'ParentID']) 
#     return results

# dataTable = saveAsTable(data)
# dataTable.shape

In [2]:
def get_text_vectors(tweets, model):
    # dataset should be a pandas dataframe
    dimension = 300
    data_array = np.empty(shape=[0, dimension])
    indexes = []
    
    for i, tweet in enumerate(tweets):
        words = tweet.split()
        if len(words) !=0:
            feature = 0
            for word in words:
                try:
                    feature += model[word]
                except:
                    pass
            feature /= len(words)
            try:
                if feature.size == dimension:  
                    data_array = np.append(data_array, [feature], axis=0)
                    indexes.append(i)
            except:
                continue
    indexes = np.asarray(indexes)
    assert indexes.size == data_array.shape[0]
    return data_array, indexes

def remove_indices(weak_signals):
    # remove indexes of tweets that do not have coverage
    indices = np.where(np.sum(weak_signals, axis=1) == -weak_signals.shape[1])[0]
    weak_signals = np.delete(weak_signals, indices, axis=0)
    
    return weak_signals

In [3]:
df = pd.read_csv('../datasets/glove.42B.300d.txt', sep=" ", quoting=3, header=None, index_col=0)
glove_model = {key: val.values for key, val in df.T.items()}

In [416]:
# create test data for the av_news data. Run once
file = 'world_first_l3'
filename = '../datasets/av-news/'+file+'/'

df = pd.read_csv(filename+file+'.csv')
np.random.seed(2000)

# set up training and test data
num_test = int(0.1*df.shape[0]) if int(0.1*df.shape[0]) < 200 else 200
test_indexes = np.random.choice(df.shape[0],num_test,replace=False)
indexes = np.arange(df.shape[0])
indexes = np.delete(indexes, test_indexes)

# get dev set
np.random.seed(2000)
dev_set = np.random.choice(indexes, 50, replace=False)
indexes = np.delete(indexes, [np.where(indexes == i)[0][0] for i in dev_set])
df.head()

,index,tweet_id,text,author,receiver,date,depth,parent_id,tokens,token_no_stop,...,topic_49,topic_50,topic_51,topic_52,topic_53,topic_54,topic_55,topic_56,topic_57,topic_assigned
0,2,893846105959661568,@WHands80 why did this move? 29th Street pic.t...,whands80,sharrowsdc,2017-08-05 10:48:13,0,89384610595966156,why did this move th street,move th street,...,0.003750,0.003743,0.001601,0.002335,0.003421,0.001901,0.012416,0.012451,0.001141,40.0
1,58,888069376687169537,I think one challenge would be depth - its rea...,whands80,bilsko,2017-07-20 12:13:33,0,88785776178194022,think one challenge would be depth its really ...,think challenge would depth shallow many stret...,...,0.001555,0.001552,0.000664,0.000968,0.001418,0.000788,0.005148,0.005162,0.000473,42.0
2,67,893935836781305856,The problem will be when an autonomous car is ...,doug_macintyre,NaN,2017-08-05 16:44:46,seed,89392287313952768,the problem will be when an autonomous car is ...,problem autonomous car accident kill,...,0.002198,0.002194,0.000938,0.001369,0.002005,0.001114,0.007278,0.007299,0.000669,47.0
3,75,893465217027461120,Humans cause most self-driving car accidents ...,valevehiclemove,NaN,2017-08-04 09:34:42,seed,89346521702746112,humans cause most self driving car accidents,human cause self driving car accident,...,0.002772,0.002767,0.001183,0.001726,0.002528,0.001405,0.009177,0.009203,0.000844,47.0
4,77,892677727479902209,Why Pittsburgh Is Still The Center Of Self-Dri...,valevehiclemove,NaN,2017-08-02 05:25:30,seed,89267772747990220,why pittsburgh is still the center of self dri...,pittsburgh center self driving car technology,...,0.002198,0.002194,0.000938,0.001369,0.002005,0.001114,0.007278,0.007299,0.000669,19.0


In [417]:
tweets = df.token_no_stop.values
tweet_vectors, tweet_index = get_text_vectors(tweets[indexes], glove_model)
dev_vectors, dev_index = get_text_vectors(tweets[dev_set], glove_model)
test_vectors, test_index = get_text_vectors(tweets[test_indexes], glove_model)
assert dev_index.size == dev_set.size
tweets.size,tweet_vectors.shape

(2169, (1919, 300))

In [418]:
# save av_news dev data
np.save(filename+'dev_features.npy', dev_vectors)

# save av_news test data
np.save(filename+'test_features.npy', test_vectors)
# np.savetxt(filename+"test_data.csv", tweets[test_indexes], delimiter=",",fmt='%s')
# np.savetxt(filename+"dev_set.csv", tweets[dev_set], delimiter=",",fmt='%s')

# save av_news data
np.save(filename+'data_features.npy', tweet_vectors)

# # incase I mistakenly resave the test tweets
# # labels = np.load(filename+'test_labels.npy')
# # num_test = np.vstack([tweets[test_indexes],labels]).T
# # np.savetxt(filename+"test_data.csv", num_test, delimiter=",",fmt='%s')
tweets = tweets[indexes]
tweets = tweets[tweet_index]

In [419]:
# load and save data labels
test = pd.read_csv(filename+'test_data.csv')
np.save(filename+'test_labels.npy', test.labels.values)

test = pd.read_csv(filename+'dev_set.csv')
np.save(filename+'dev_labels.npy', test.labels.values)

In [373]:
# find url in tweets
def urlInTweet(raw_texts):
    weak_signal = []
    for tweet in raw_texts:
        urls = re.findall('(?:(?:https?|ftp):\/\/)?[\w/\-?=%.]+\.[\w/\-?=%.]+', tweet)
        x = 1 if len(urls) > 0 else 0
        weak_signal.append(x)
    return weak_signal

test=df.text.values[indexes]
url_signal = urlInTweet(test[tweet_index])

In [374]:
# from scipy import spatial
# # keyword weak_signal
# keywords = ['self','autonomous','crash']
# keywords_vec = [glove_model[key] for key in keywords]
# keywords_vec = np.mean(keywords_vec)
# keyword_signal = [1 - spatial.distance.cosine(vec, keywords_vec) for vec in tweet_vectors] 

In [375]:
# define cluster signals
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=10, init='k-means++').fit(tweet_vectors)
cluster_signals = np.ones(tweets.size)

In [386]:
#fi_av [0,6,8],uni_mich [1,5,6],goo_lex [1,6,8],int_tesla [0,3,7],mcity_shu [0,1,6,8],tes_feb [0,1,2,4,5,8],wor_l3 [0,3,4]
negative_samples = [0,3,4] # change this as you investigate the data
tweets[kmeans.labels_==4]

array(['indifferent mainstream tesla weakest spot exterior design',
       'morality question fascinating autonomous car hit jaywalker swerve endanger driver',
       'scared killed jaywalking pedestrian',
       'grrrr glad hurt injured driver slow check switch riding dwn boylston bbay regular event',
       'blame pissed put blame delta insult passenger',
       'crash tesla promoted autopilot china zi dong jia shi meaning car drive',
       'elon musk experienced zuckerberg autopilot car killed person scary regulated',
       'gostei de um autopilot predicts car crash happen',
       'elon interior shared noticed', 'think government study elon musk',
       'perjurous gnome job',
       'uber degrades city professionalism driving occupation',
       'room cross almighty mercy endures',
       'fact joy used demetrius shipp pac interesting',
       'trying virgin airline shitcarts road',
       'problem automate retweets verbal orgasm',
       'know way careless pedestrian mow kill p

In [387]:
for num in negative_samples:
    cluster_signals[cluster_signals==num] = 0

In [388]:
# Define weak signals for seed tweets and topic signals
seed_signals=df.depth.values[indexes]
seed_signals=1*(seed_signals[tweet_index] ==' seed')

# first av crash, change the topics for a different dataset
av_topics = []
for line in open(filename+file+'.txt'):
    lines = line.split(' ')
    if 'AV' in lines[1]:
        av_topics.append(lines[0])

In [389]:
# add up the topics to get topic weak signal
col_name='topic_'
topic_signal = np.zeros(tweet_vectors.shape[0])
for num in av_topics:
    topic = df[col_name+num][indexes].values
    topic_signal = np.add(topic_signal, topic[tweet_index])

In [390]:
# join the seed and topic signals
weak_signals = np.stack([seed_signals, topic_signal, url_signal, cluster_signals], axis=1)
weak_signals.shape

(1919, 4)

In [391]:
# save the av news weak signals
np.save(filename+'weak_signals.npy', weak_signals)

# Evaluation

In [66]:
def load_results(filename):
    with open(filename, 'r') as content:
        data = json.load(content)
    return data

# read from the train data
all_signal_results = load_results('results/av_news/crash_results.json')
# # load indexes of original data Table
# data_indexes = np.load('../datasets/av-news/indexes.npy')

# Get predictions of the models
train_indexes = np.ravel(all_signal_results['Adversarial model']['train_indexes'])
train_pred = np.ravel(all_signal_results['Adversarial model']['train_predictions'])
majority_vote = np.ravel(all_signal_results['Weak model']['baseline_train_predictions'])

In [67]:
# I only care about the positive predictions from the models
train_onespred = train_indexes[train_pred==1.0]
majority_onespred = train_indexes[majority_vote==1.0]

In [68]:
# sample 100 predictions to manually evaluate
indexes = np.random.choice(train_onespred, 100, replace=False)
#sample 100 predictions from majority vote
majority_indexes = np.random.choice(majority_onespred, 100, replace=False)

# Create a Pandas Excel writer using XlsxWriter as the engine.
writer = pd.ExcelWriter('results/av_news/sample_pred.xlsx', engine='xlsxwriter')
df.tokens[indexes].to_excel(writer, sheet_name = 'rcl_pred')
df.tokens[majority_indexes].to_excel(writer, sheet_name = 'majority_pred')
random_sample = np.random.choice(train_indexes, 100, replace=False)
df.tokens[random_sample].to_excel(writer, sheet_name = 'random')
writer.save()
writer.close()

In [69]:
# randomnly sample 100 tweets from the data
pprint(df.tokens.values[random_sample])

array(['driver in nj claims that his tesla was in autopilot mode and would not let him take control of the steering wheel to avoid an accident',
       'you might be able to kill some off but do not know how you would change it without medical intervention',
       'waymo cruise apple and other autonomous vehicle makers reveal their driving',
       'drop in sales in jan was expected usual slow month for industry rush to buy in dec for tax credit production diverted to europe china for jan feb agree we will see where demand levels off but definitely not panicking over usually slow month',
       'ai is our future just whose ai love it refer to myself as ai because it is slightly better than fairing princess',
       'think it is just hong kong that does it as it is based around the mountain there',
       'impossible now thanks to drive by wire not impossible when it was simple mechanical system our bmw series back in the day stuck at mph on the motorway the dealership said we imagined